In [ ]:
import tensorflow as tf 
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/Data/IDMT_full_melspec_final.csv")

In [ ]:
arr=np.load("/content/drive/MyDrive/Data/features_full_melspec_final.npy")

In [ ]:
df['vehicle'].value_counts()

None    4071
C       3902
T        268
M        246
Name: vehicle, dtype: int64

In [ ]:
X=arr
y=np.array(df['vehicle'].tolist())
### Label Encoding -> Label Encoder
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))
### Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=0)

In [ ]:
print(np.shape(X_train))
print(np.shape(X_test))
print(np.shape(y_train))
print(np.shape(y_test))

(5940, 128, 87)
(2547, 128, 87)
(5940, 4)
(2547, 4)


In [ ]:
from tensorflow.keras import datasets, layers, models,Sequential
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten,Conv3D,MaxPool1D,Conv2D,Conv1D,MaxPool2D
from tensorflow.keras.optimizers import Adam
from sklearn import metrics
from keras.callbacks import ReduceLROnPlateau
from sklearn.metrics import roc_curve,roc_auc_score
import matplotlib.pyplot as plt

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='accuracy', patience = 2, verbose=1,factor=0.5, min_lr=0.00001)

In [ ]:
nn = models.Sequential([
        layers.Conv1D(input_shape=(128,87),filters=64,kernel_size=3,padding='same', activation='relu'),
        layers.MaxPool1D(pool_size=2),
        layers.Dropout(0.2),
        layers.Conv1D(filters=64,kernel_size=3,padding='same', activation='relu'),
        layers.MaxPool1D(pool_size=2),
        layers.Dropout(0.2),
        layers.Conv1D(filters=32,kernel_size=3,padding='same', activation='relu'),
        layers.MaxPool1D(pool_size=3),
        layers.Dropout(0.2),
        layers.Conv1D(filters=16,kernel_size=3,padding='same', activation='relu'),
        layers.MaxPool1D(pool_size=2),
        layers.Flatten(),
        layers.Dense(1000, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(500, activation='relu'),
        layers.Dense(100, activation='relu'),
        layers.Dense(4, activation='softmax')    
    ])

In [ ]:
nn.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history=nn.fit(X_train, y_train, epochs=30,callbacks = [learning_rate_reduction])

Epoch 1/30
186/186 [==============================] - 16s 8ms/step - loss: 0.3840 - accuracy: 0.9098 - lr: 0.0010
Epoch 2/30
186/186 [==============================] - 1s 6ms/step - loss: 0.1432 - accuracy: 0.9512 - lr: 0.0010
Epoch 3/30
186/186 [==============================] - 1s 6ms/step - loss: 0.1350 - accuracy: 0.9567 - lr: 0.0010
Epoch 4/30
186/186 [==============================] - 1s 6ms/step - loss: 0.1135 - accuracy: 0.9613 - lr: 0.0010
Epoch 5/30
186/186 [==============================] - 1s 6ms/step - loss: 0.0935 - accuracy: 0.9685 - lr: 0.0010
Epoch 6/30
186/186 [==============================] - 1s 6ms/step - loss: 0.1062 - accuracy: 0.9633 - lr: 0.0010
Epoch 7/30
186/186 [==============================] - 1s 6ms/step - loss: 0.0992 - accuracy: 0.9690 - lr: 0.0010
Epoch 8/30
186/186 [==============================] - 1s 6ms/step - loss: 0.0763 - accuracy: 0.9737 - lr: 0.0010
Epoch 9/30
186/186 [==============================] - 1s 6ms/step - loss: 0.0639 - accuracy: 0.

In [ ]:
print("Accuracy of the model on Training Data is - " , nn.evaluate(X_train,y_train)[1]*100 , "%")
print("Accuracy of the model on Testing Data is - " , nn.evaluate(X_test,y_test)[1]*100 , "%")

186/186 [==============================] - 1s 4ms/step - loss: 0.0124 - accuracy: 0.9961
Accuracy of the model on Training Data is -  99.61279630661011 %
80/80 [==============================] - 1s 7ms/step - loss: 0.1193 - accuracy: 0.9808
Accuracy of the model on Testing Data is -  98.0761706829071 %


In [ ]:
import sklearn.metrics as skm

# y_predict=nn.predict(X_test)
y_predict=(nn.predict(X_test) > 0.5).astype("int32")
print(y_predict[0:2])
y_true=y_test
print(y_true)

# res = tf.math.confusion_matrix(y_true,y_predict)
cm = skm.multilabel_confusion_matrix(y_true, y_predict)
print(cm)
print( skm.classification_report(y_true,y_predict))

[[0 0 1 0]
 [0 0 1 0]]
[[0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 ...
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
[[[1356   24]
  [  15 1152]]

 [[2468    8]
  [   3   68]]

 [[1320    4]
  [  16 1207]]

 [[2449   12]
  [  15   71]]]
              precision    recall  f1-score   support

           0       0.98      0.99      0.98      1167
           1       0.89      0.96      0.93        71
           2       1.00      0.99      0.99      1223
           3       0.86      0.83      0.84        86

   micro avg       0.98      0.98      0.98      2547
   macro avg       0.93      0.94      0.94      2547
weighted avg       0.98      0.98      0.98      2547
 samples avg       0.98      0.98      0.98      2547



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
